In [1]:
import pandas as pd
import numpy as np
import itertools, csv

massWidth = 30 #GeV

In [2]:
# *** 0. Import Dataset
qcd_raw = pd.read_csv('../higgsReconstruction/EventPlotting/qcd_outputDataForLearning.csv')
hh_raw = pd.read_csv('../higgsReconstruction/EventPlotting/dihiggs_outputDataForLearning.csv')

hh_raw.head()
print(len(hh_raw), "rows of dihiggs data")
#print(hh_raw.columns)
qcd_raw.head()
print(len(qcd_raw), "rows of qcd data")

#variableNames = ['hh_mass', 'h1_mass', 'h2_mass']
#variableNames = ['deltaR(h1, h2)', 'deltaR(h1 jets)', 'deltaR(h2 jets)']
variableNames = ['hh_mass', 'h1_mass', 'h2_mass', 'deltaR(h1, h2)', 'deltaR(h1 jets)', 'deltaR(h2 jets)']

hh_reducedData  = hh_raw[variableNames]
qcd_reducedData = qcd_raw[variableNames]
print(hh_reducedData.columns, variableNames[0])
hh_reducedData.hist(column=variableNames[2], bins=100)
qcd_reducedData.hist(column=variableNames[2], bins=100)

type(hh_reducedData)

4605 rows of dihiggs data
1703 rows of qcd data
Index(['hh_mass', 'h1_mass', 'h2_mass', 'deltaR(h1, h2)', 'deltaR(h1 jets)',
       'deltaR(h2 jets)'],
      dtype='object') hh_mass


pandas.core.frame.DataFrame

In [3]:
def returnCutValueByConstantEfficiency( _variable, _signal, _background, _eff, _inequality = '>'):
    """return a cut value based on keeping some constant efficiency of signal"""

    _bestCutValue = -1
    _nTotalSignal =len(_signal) 
    _nTotalBackground =len(_background) 
    _cuts = []
    
    _minVal = int(min(min(_background), min(_signal))) if 'mass' not in _variable else int(min(min(_background), min(_signal))) - int(min(min(_background), min(_signal)))%5
    _maxVal = int(max(max(_background), max(_signal))) if 'mass' not in _variable else int(max(max(_background), max(_signal))) - int(max(max(_background), max(_signal)))%5
    if 'mass' in _variable:
        #_cuts = list(range(_minVal, _maxVal, _stepSize))
        _cuts = list(range(0, _maxVal, 5))
        #print(_maxVal, max(_background), max(_signal))
    elif 'deltaR' in _variable:
        #_cuts = np.linspace(_minVal, _maxVal, 100)
        _cuts = np.linspace(0, 5, 101)


    for iCutValue in _cuts:
        if _inequality == '<':
            _nSignal = sum( value < iCutValue for value in _signal)
            _nBackground = sum( value < iCutValue for value in _background)
        elif _inequality == '>':
            _nSignal = sum( value > iCutValue for value in _signal)
            _nBackground = sum( value > iCutValue for value in _background)
        else:
            print("Unknown inequality operator {0}. EXITING".format(_inequality))
            return _bestCutValue, _nTotalSignal, _nTotalBackground
        
        # safety check to avoid division by 0
        if _nBackground == 0:
            continue
        
        if _inequality == '<':
            if _nSignal / _nTotalSignal >= _eff:
                _bestCutValue = iCutValue
                return _bestCutValue, _nSignal, _nBackground
        elif _inequality== '>':
            if _nSignal / _nTotalSignal < _eff: # passed threshold so return previous cut
                _nSignalBest = sum( value > _bestCutValue for value in _signal)
                _nBackgroundBest = sum( value > _bestCutValue for value in _background)
                return _bestCutValue, _nSignalBest, _nBackgroundBest
            else:
                _bestCutValue = iCutValue
            
    return _bestCutValue, -1, -1
    
    
def returnBestCutValue( _variable, _signal, _background, _method='S/B'):
    """find best cut according to user-specified significance metric"""
    
    _bestSignificance = -1
    _bestCutValue = -1
    _massWidth = massWidth #GeV
    _nTotalSignal =len(_signal) 
    _nTotalBackground =len(_background) 
    _cuts = []
    
    _minVal = int(min(min(sortedBackground), min(sortedSignal))) if 'mass' not in _variable else int(min(min(sortedBackground), min(sortedSignal))) - int(min(min(sortedBackground), min(sortedSignal)))%5
    _maxVal = int(max(max(sortedBackground), max(sortedSignal))) if 'mass' not in _variable else int(max(max(sortedBackground), max(sortedSignal))) - int(max(max(sortedBackground), max(sortedSignal)))%5
    if 'mass' in _variable:
        _stepSize = 0.05 if 'mass' not in _variable else 5
        _cuts = list(range(_minVal, _maxVal, _stepSize))
    else:
        _cuts = np.linspace(_minVal, _maxVal, 100)
    
    for iCutValue in _cuts:
        if 'mass' in _variable:
            _nSignal = sum( (value > iCutValue and value < (iCutValue+_massWidth)) for value in _signal) 
            _nBackground = sum( (value > iCutValue and value < (iCutValue+_massWidth)) for value in _background)
            #_nSignal = sum( (value > iCutValue ) for value in _signal)
            #_nBackground = sum( (value > iCutValue) for value in _background)
        else:
            _nSignal = sum( value < iCutValue for value in _signal)
            _nBackground = sum( value < iCutValue for value in _background)

        # temporary fix since samples with different number of events
        #_nSignal = _nSignal / _nTotalSignal
        #_nBackground = _nBackground / _nTotalBackground
        _nSignal = _nSignal * (_nTotalBackground / _nTotalSignal )
        #_nBackground = _nBackground / _nTotalBackground
        
        # safety check to avoid division by 0
        if _nBackground == 0:
            continue
        
        #if _method == 'S/sqrt(B)':
        #    print(_nSignal, _nBackground, iCutValue, (_nSignal / np.sqrt(_nBackground)), (_nSignal / np.sqrt(_nSignal + _nBackground)))
        
        if _method == 'S/B' and (_nSignal / _nBackground) > _bestSignificance:
            _bestSignificance = (_nSignal / _nBackground)
            _bestCutValue = iCutValue
        elif _method == 'S/sqrt(B)' and (_nSignal / np.sqrt(_nBackground)) > _bestSignificance:
            _bestSignificance = (_nSignal / np.sqrt(_nBackground))
            _bestCutValue = iCutValue
        elif _method == 'S/sqrt(S+B)' and (_nSignal / np.sqrt(_nSignal + _nBackground)) > _bestSignificance:
            _bestSignificance = (_nSignal / np.sqrt(_nSignal + _nBackground))
            _bestCutValue = iCutValue
        
    return _bestSignificance, _bestCutValue


def returnSignificanceOrderedCutDict( _method, _varNames, _signalDataFrame, _backgroundDataFrame):
    """function to return list of cuts ordered by descending significance"""
    
    _orderedVariableAndCutDict = {}
    _unprocessedVariables = _varNames
    _signalAfterCuts = _signalDataFrame
    _backgroundAfterCuts = _backgroundDataFrame
    
    while len(_unprocessedVariables)>0:
        _iBestCut = -1
        _iBestSignificance = -1
        _iBestVariable = ''
        print('iteration {0}, signal has {1} rows'.format(len(_unprocessedVariables), len(_signalAfterCuts)))
        print('iteration {0}, background has {1} rows'.format(len(_unprocessedVariables), len(_backgroundAfterCuts)))
        
        for iVariable in _unprocessedVariables:
            _sortedSignal = np.sort(_signalDataFrame[iVariable].values)
            _sortedBackground = np.sort(_backgroundDataFrame[iVariable].values)
            #print(_sortedSignal)
            _tempSignificance, _tempCut = returnBestCutValue( iVariable, _sortedSignal, _sortedBackground, _method)
            #print ( iVariable, _tempSignificance, _tempCut )
                
            # most significant 1D variable so far in this iteration
            if _tempSignificance > _iBestSignificance:
                _iBestSignificance = _tempSignificance
                _iBestCut = _tempCut
                _iBestVariable = iVariable
        
        print('Iteration {0} chose variable {1} with significance {2} at cut {3}'.format(int(len(_varNames)-len(_unprocessedVariables)), _iBestVariable, _iBestSignificance, _iBestCut))
        _unprocessedVariables.remove(_iBestVariable)
        _orderedVariableAndCutDict[_iBestVariable] = [_iBestCut, _iBestSignificance]
        if 'mass' in _iBestVariable:
            _signalAfterCuts = _signalAfterCuts[ (_signalAfterCuts[_iBestVariable] > _iBestCut) & (_signalAfterCuts[_iBestVariable]< (_iBestCut + massWidth))]
            _backgroundAfterCuts = _backgroundAfterCuts[ (_backgroundAfterCuts[_iBestVariable] > _iBestCut) & (_backgroundAfterCuts[_iBestVariable]< (_iBestCut + massWidth))]
        else:
            _signalAfterCuts = _signalAfterCuts[ _signalAfterCuts[_iBestVariable] < _iBestCut ]
            _backgroundAfterCuts = _backgroundAfterCuts[ _backgroundAfterCuts[_iBestVariable] < _iBestCut ]
            
    return _orderedVariableAndCutDict

In [4]:
for iColumn in range(0, len(hh_reducedData.columns) ):
    varName = variableNames[iColumn]
    sortedSignal = np.sort(hh_reducedData[varName].values)
    sortedBackground = np.sort(qcd_reducedData[varName].values)
    
    bestCut, significance = returnBestCutValue( varName, sortedSignal, sortedBackground, 'S/B')
    print ( varName, bestCut, significance )
    bestCut, significance = returnBestCutValue( varName, sortedSignal, sortedBackground, 'S/sqrt(B)')
    print ( varName, bestCut, significance )
    bestCut, significance = returnBestCutValue( varName, sortedSignal, sortedBackground, 'S/sqrt(S+B)')
    print ( varName, bestCut, significance )
    
print("=====================================")
    

hh_mass 3.5813703640208012 190
hh_mass 17.18855249526857 280
hh_mass 10.879546612581514 280
h1_mass 3.1514705188122547 85
h1_mass 48.27333751421789 90
h1_mass 23.868509420973616 90
h2_mass 2.4207239735733084 85
h2_mass 41.57729962477844 85
h2_mass 22.48004951856836 85
deltaR(h1, h2) 3.6981541802388707 0.2828282828282828
deltaR(h1, h2) 44.24970291081079 3.323232323232323
deltaR(h1, h2) 29.482700944781083 3.818181818181818
deltaR(h1 jets) 1.5422089772911034 1.9696969696969697
deltaR(h1 jets) 43.241146858604495 2.5757575757575757
deltaR(h1 jets) 29.601200756726165 3.2323232323232323
deltaR(h2 jets) 1.1937424155329883 2.5757575757575757
deltaR(h2 jets) 42.825355396766525 3.3333333333333335
deltaR(h2 jets) 29.48199235448601 3.4343434343434343


In [5]:
orderedCuts_SoverB = returnSignificanceOrderedCutDict( 'S/B', variableNames.copy(), hh_reducedData.copy(), qcd_reducedData.copy())
print (orderedCuts_SoverB)
orderedCuts_SoverSqrtB = returnSignificanceOrderedCutDict( 'S/sqrt(B)', variableNames.copy(), hh_reducedData.copy(), qcd_reducedData.copy())
print (orderedCuts_SoverSqrtB)
orderedCuts_SoverSqrtSB = returnSignificanceOrderedCutDict( 'S/sqrt(S+B)', variableNames.copy(), hh_reducedData.copy(), qcd_reducedData.copy())
print (orderedCuts_SoverSqrtSB)

iteration 6, signal has 4605 rows
iteration 6, background has 1703 rows
Iteration 0 chose variable deltaR(h1, h2) with significance 3.205066956207021 at cut 0.45454545454545453
iteration 5, signal has 26 rows
iteration 5, background has 3 rows
Iteration 0 chose variable deltaR(h1 jets) with significance 1.5422089772911034 at cut 1.9696969696969697
iteration 4, signal has 17 rows
iteration 4, background has 2 rows
Iteration 0 chose variable deltaR(h2 jets) with significance 1.1937424155329883 at cut 2.5757575757575757
iteration 3, signal has 14 rows
iteration 3, background has 2 rows
Iteration 0 chose variable h1_mass with significance 1.0566154800682488 at cut 0
iteration 2, signal has 0 rows
iteration 2, background has 0 rows
Iteration 0 chose variable h2_mass with significance 0.25602605863192185 at cut 0
iteration 1, signal has 0 rows
iteration 1, background has 0 rows
Iteration 0 chose variable  with significance -1 at cut -1


ValueError: list.remove(x): x not in list

In [6]:
def getCutsForSpecifiedEfficiency( _signal, _background, _eff, _inequality = '<'):
    """get cuts for all variables given user-specified efficency"""
    variablesAndCuts_ = {}
    
    print("========== Efficiency {0}% , Using {1} =========".format(_eff, _inequality) )
    for iColumn in range(0, len(_signal.columns) ):
        varName = variableNames[iColumn]
        sortedSignal = np.sort(_signal[varName].values)
        sortedBackground = np.sort(_background[varName].values)
    
        cutVal, nSig, nBkg = returnCutValueByConstantEfficiency( varName, sortedSignal, sortedBackground, _eff, _inequality)
        print('Cut of {4} {0} on {1} yields nSig = {2} and nBkg = {3}'.format(round(cutVal,2), varName, nSig, nBkg, _inequality))    
        variablesAndCuts_[varName] = round(cutVal,2)
    
    return variablesAndCuts_


def returnNumberSignalAndBackgroundAfterCuts( _signal, _background, _cuts, _useVariableString = '', _inequality = '>', _significanceMetric = 'S/sqrt(B)'):
    """return number of signal and background (and maybe some significance score?) for a passed set of cuts"""
    
    _unprocessedVariables = [x for x in _cuts.keys() if _useVariableString in x] #list(_cuts.keys())
    _signalAfterCuts = _signal
    _backgroundAfterCuts = _background
    _nTotalSignal = len(_signalAfterCuts)
    _nTotalBackground = len(_backgroundAfterCuts)
    
    while len(_unprocessedVariables)>0:
        print('iteration {0}, signal has {1} rows'.format(len(_unprocessedVariables), len(_signalAfterCuts)))
        print('iteration {0}, background has {1} rows'.format(len(_unprocessedVariables), len(_backgroundAfterCuts)))
        
        for iVariable in _unprocessedVariables:
            _cutValue = _cuts[iVariable]
            _sortedSignal = np.sort(_signalAfterCuts[iVariable].values)
            _sortedBackground = np.sort(_backgroundAfterCuts[iVariable].values)

            _signalAfterCuts = _signalAfterCuts[ (_signalAfterCuts[iVariable] > _cutValue)]
            _backgroundAfterCuts = _backgroundAfterCuts[ (_backgroundAfterCuts[iVariable] > _cutValue)]

            _nSignal = len(_signalAfterCuts) * (_nTotalBackground / _nTotalSignal )
            _nBackground = len(_backgroundAfterCuts) 
            print('Iteration {0} chose variable {1} with N_signal = {2} ({4}) and N_background = {3} ({5})'.format(int(len(_unprocessedVariables)), iVariable, len(_signalAfterCuts), len(_backgroundAfterCuts), round(_nSignal,1), _nBackground))
            _unprocessedVariables.remove(iVariable)
    
    _nSignal = len(_signalAfterCuts) * (_nTotalBackground / _nTotalSignal )
    _nBackground = len(_backgroundAfterCuts) 
    print('{0} = {1}'.format(_significanceMetric, round( _nSignal / np.sqrt(_nBackground), 2)))
    
    return 

In [7]:
#getCutsForSpecifiedEfficiency(hh_reducedData, qcd_reducedData, 0.80, '<')
#getCutsForSpecifiedEfficiency(hh_reducedData, qcd_reducedData, 0.85, '<')
#getCutsForSpecifiedEfficiency(hh_reducedData, qcd_reducedData, 0.90, '<')

cutsFor80PercentEfficiency = getCutsForSpecifiedEfficiency(hh_reducedData.copy(), qcd_reducedData.copy(), 0.80, '>')
cutsFor85PercentEfficiency = getCutsForSpecifiedEfficiency(hh_reducedData.copy(), qcd_reducedData.copy(), 0.85, '>')
cutsFor90PercentEfficiency = getCutsForSpecifiedEfficiency(hh_reducedData.copy(), qcd_reducedData.copy(), 0.90, '>')

========== Efficiency 0.8% , Using > =========
Cut of > 270 on hh_mass yields nSig = 3733 and nBkg = 1525
Cut of > 85 on h1_mass yields nSig = 3775 and nBkg = 1242
Cut of > 75 on h2_mass yields nSig = 3810 and nBkg = 1161
Cut of > 2.1 on deltaR(h1, h2) yields nSig = 3722 and nBkg = 1567
Cut of > 0.95 on deltaR(h1 jets) yields nSig = 3733 and nBkg = 1329
Cut of > 1.1 on deltaR(h2 jets) yields nSig = 3709 and nBkg = 1229
========== Efficiency 0.85% , Using > =========
Cut of > 250 on hh_mass yields nSig = 3974 and nBkg = 1581
Cut of > 80 on h1_mass yields nSig = 3963 and nBkg = 1275
Cut of > 70 on h2_mass yields nSig = 4001 and nBkg = 1205
Cut of > 1.85 on deltaR(h1, h2) yields nSig = 3916 and nBkg = 1598
Cut of > 0.85 on deltaR(h1 jets) yields nSig = 3923 and nBkg = 1374
Cut of > 1.0 on deltaR(h2 jets) yields nSig = 3919 and nBkg = 1284
========== Efficiency 0.9% , Using > =========
Cut of > 230 on hh_mass yields nSig = 4165 and nBkg = 1617
Cut of > 70 on h1_mass yields nSig = 4217 and 

In [8]:
returnNumberSignalAndBackgroundAfterCuts( hh_reducedData.copy(), qcd_reducedData.copy(), cutsFor90PercentEfficiency, 'mass')
returnNumberSignalAndBackgroundAfterCuts( hh_reducedData.copy(), qcd_reducedData.copy(), cutsFor90PercentEfficiency, 'elta')
returnNumberSignalAndBackgroundAfterCuts( hh_reducedData.copy(), qcd_reducedData.copy(), cutsFor90PercentEfficiency)

iteration 3, signal has 4605 rows
iteration 3, background has 1703 rows
Iteration 3 chose variable hh_mass with N_signal = 4165 (1540.3) and N_background = 1617 (1617)
Iteration 2 chose variable h2_mass with N_signal = 3959 (1464.1) and N_background = 1271 (1271)
iteration 1, signal has 3959 rows
iteration 1, background has 1271 rows
Iteration 1 chose variable h1_mass with N_signal = 3798 (1404.6) and N_background = 1123 (1123)
S/sqrt(B) = 41.91
iteration 3, signal has 4605 rows
iteration 3, background has 1703 rows
Iteration 3 chose variable deltaR(h1, h2) with N_signal = 4163 (1539.5) and N_background = 1637 (1637)
Iteration 2 chose variable deltaR(h2 jets) with N_signal = 3787 (1400.5) and N_background = 1304 (1304)
iteration 1, signal has 3787 rows
iteration 1, background has 1304 rows
Iteration 1 chose variable deltaR(h1 jets) with N_signal = 3531 (1305.8) and N_background = 1182 (1182)
S/sqrt(B) = 37.98
iteration 6, signal has 4605 rows
iteration 6, background has 1703 rows
Itera

In [ ]:
4165*1703/4605

In [ ]:
a= pd.DataFrame()

In [ ]:
type(a)